<a href="https://colab.research.google.com/github/Artmagic/GP2_team34/blob/main/%D0%9E%D0%91%D0%A9%D0%95%D0%9F%D0%98%D0%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests
import pandas as pd

url = "https://apidata.mos.ru/v1/datasets"

response = requests.get(url, params={"$top": 2000, "api_key": "c24b8285-dabc-426c-8e6c-14fde6952d63"})
data = response.json()

items = pd.DataFrame(data["Items"])
items.head()

,CategoryId,PublishDate,Keywords,IdentificationNumber,ContainsGeodata,IsSeasonal,IsNew,VersionNumber,Caption,ContainsAccEnvData,IsForeign,IsArchive,SefUrl,Id,ReleaseNumber,DepartmentId,Season,FullDescription,LastUpdateDate
0,107,2014-05-13,"библиотеки, выставка, выставочные залы, дворец...",7702155262-culturehouse,True,False,False,3,Дома культуры и клубы,True,False,False,None,493,1094,1008,0,<p>Набор данных содержит информацию о домах ку...,2025-10-30
1,111,2014-04-10,"библиотеки, выставочные залы, дома культуры и ...",7702155262-MovieTheaters,True,False,False,3,Кинотеатры,True,False,False,None,495,58,1008,0,"<p style=""margin-left:0cm; margin-right:0cm"">Н...",2025-10-30
2,111,2014-01-15,"аттракцион, аттракционы",7709083955-attractions,True,False,False,7,Аттракционы в парках и на площадках,False,False,False,None,498,553,1214,0,<!--[if gte mso 9]><xml>\n <o:OfficeDocumentSe...,2025-10-30
3,112,2014-04-03,"государственные учреждения, государственное уч...",7701236617-DZKHIBgos,True,False,False,3,"Организации, находящиеся в ведомственном подчи...",False,False,False,None,499,26,1006,0,"<p><span style=""font-family:Tahoma; font-size:...",2024-12-06
4,112,2014-04-03,"инженерные службы, инженерная служба, службы, ...",7701236617-Regionalengineeringservices,True,False,False,3,Государственные казенные учреждения «Инженерны...,True,False,True,None,500,30,1006,0,"<p style=""margin-bottom:0cm;margin-bottom:.000...",2022-01-21


In [21]:
items[items["Caption"].str.contains("питание", case=False, na=False)][["Id", "Caption"]]

,Id,Caption
415,1903,Общественное питание в Москве


In [22]:
items[items["Caption"].str.contains("кафе", case=False, na=False)][["Id", "Caption"]]

,Id,Caption
79,587,Сезонные кафе при стационарных предприятиях об...
347,1792,Кафе
349,1794,Кафетерии
832,3245,Сезонные кафе при стационарных предприятиях об...
999,61360,Сезонные кафе при стационарных предприятиях об...


In [23]:
import pandas as pd
import time

base_url = f"https://apidata.mos.ru/v1/datasets/1903/rows"
page_size = 1000
skip = 0
all_records = []

while True:
    params = {
        "api_key": "c24b8285-dabc-426c-8e6c-14fde6952d63",
        "$top": page_size,
        "$skip": skip
    }
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Ошибка {response.status_code} при skip={skip}")
        break

    batch = response.json()
    if not batch:
        break

    # Извлекаем "Cells"
    all_records.extend([b["Cells"] for b in batch])

    skip += page_size

    # Небольшая пауза, чтобы не перегружать API
    time.sleep(0.5)

df = pd.DataFrame(all_records)
print("Итоговый размер:", df.shape)


Итоговый размер: (22820, 15)


In [24]:
if "PublicPhone" in df.columns:
    df["PublicPhone"] = df["PublicPhone"].apply(
        lambda x: x[0]["PublicPhone"] if isinstance(x, list) and x else None
    )
df

,ID,Name,global_id,IsNetObject,OperatingCompany,TypeObject,AdmArea,District,Address,PublicPhone,SeatsCount,SocialPrivileges,Longitude_WGS84,Latitude_WGS84,geoData
0,000077874,Родник,637376331,нет,,кафе,Центральный административный округ,Таганский район,"город Москва, улица Талалихина, дом 2/1, корпус 1",(495) 676-55-35,35,нет,37.6733061300344000,55.7382386551547000,"{'coordinates': [37.67330613, 55.738238655], '..."
1,000024309,Кафе «Академия»,637376349,нет,,ресторан,Центральный административный округ,Таганский район,"Российская Федерация, город Москва, внутригоро...",(495) 662-30-10,95,нет,37.6696475969381000,55.7355114718314000,"{'coordinates': [37.669647597, 55.735511472], ..."
2,000027429,ГБОУ «Школа № 1430 имени Героя Социалистическо...,637376480,нет,,столовая,Северо-Восточный административный округ,район Лианозово,"Российская Федерация, город Москва, внутригоро...",(499) 908-06-15,240,нет,37.5669400000000020,55.9040188006692200,"{'coordinates': [37.56694, 55.904018801], 'typ..."
3,00148815,Брусника,637376481,да,Брусника,кафе,Центральный административный округ,район Арбат,"Российская Федерация, город Москва, внутригоро...",(495) 697-04-89,10,нет,37.5980890000000020,55.7474487549306520,"{'coordinates': [37.598089, 55.747448755], 'ty..."
4,000023394,Хинкальная Гогиели,637376678,нет,,кафе,Юго-Восточный административный округ,район Лефортово,"Российская Федерация, город Москва, внутригоро...",(495) 361-38-50,24,нет,37.7199503788590680,55.7492759892765620,"{'coordinates': [37.719950379, 55.749275989], ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22815,00571438,Масала Хаус индийская кухня,2824485162,нет,,кафе,Юго-Восточный административный округ,район Выхино-Жулебино,"Российская Федерация, город Москва, внутригоро...",(977) 620-97-95,20,нет,37.8519430000000000,55.6867870000000000,"{'coordinates': [37.851943, 55.686787], 'type'..."
22816,00571451,GREEN TERRA,2824485169,нет,,предприятие быстрого обслуживания,Западный административный округ,район Кунцево,"Российская Федерация, город Москва, внутригоро...",(929) 500-00-76,1,нет,37.4016440000000000,55.7437450000000000,"{'coordinates': [37.401644, 55.743745], 'type'..."
22817,00571463,NOBA кофейня бистро кондитерская,2824485176,нет,,кафе,Южный административный округ,район Чертаново Южное,"Российская Федерация, город Москва, внутригоро...",(967) 183-36-06,15,нет,37.6027140000000000,55.5892030000000000,"{'coordinates': [37.602714, 55.589203], 'type'..."
22818,00571482,Surf coffee,2824485182,да,Surf Coffee,кафетерий,Юго-Восточный административный округ,Южнопортовый район,"Российская Федерация, город Москва, внутригоро...",(495) 600-95-27,8,нет,37.6906860000000000,55.7201390000000000,"{'coordinates': [37.690686, 55.720139], 'type'..."


In [25]:
df.to_csv("food_moscow_full.csv", index=False, encoding="utf-8-sig")